# NeuralNetwork.

In [0]:
# imports.
import pandas as pd
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt

In [2]:
# read in the data.
df = pd.read_csv('https://raw.githubusercontent.com/spotify-song-suggester-1/spotify-app/master/ds/Data/SpotifyTracks_doubleforloop_genre_year.csv',index_col=[0])
# show the data frame shape.
print(df.shape)
# show the data frame with headers.
df.head()

(164449, 20)


,artist_name,track_name,track_id,popularity,year,genre,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Gorillaz,On Melancholy Hill,0q6LuUqGLUiCPP1cbdwFs3,75,2010,alternative,0,0.000015,0.689,233867,0.739,0.509000,2,0.0640,-5.810,1,0.0260,120.423,4,0.578
1,Avenged Sevenfold,Nightmare,4UEo1b0wWrtHMC8bVqPiH8,70,2010,alternative,1,0.000318,0.554,374453,0.949,0.000100,2,0.2000,-4.928,1,0.0787,129.984,4,0.233
2,The Black Keys,Howlin' for You,0grFc6klR3hxoHLcgCYsF4,66,2010,alternative,2,0.028000,0.705,191800,0.735,0.078300,11,0.1120,-6.646,1,0.0931,132.627,4,0.448
3,My Darkest Days,Porn Star Dancing,3Q8zopc4ABXhysDb1sgLVW,65,2010,alternative,3,0.013900,0.477,199013,0.917,0.000000,2,0.0756,-3.399,1,0.0837,160.044,4,0.271
4,Volbeat,A Warrior's Call,0hTiTU0yqthnByyZDD3bcc,62,2010,alternative,4,0.000750,0.374,263080,0.903,0.000006,10,0.2440,-4.490,1,0.0825,109.118,3,0.429


In [3]:
# new data frame with encoded genre column.
genreframe = pd.concat([df,pd.get_dummies(df['genre'], prefix='genre')],axis=1)
# show the data frame with headers.
print(genreframe.shape)
genreframe.head()

(164449, 32)


,artist_name,track_name,track_id,popularity,year,genre,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,genre_alternative,genre_country,genre_dance,genre_folk,genre_grunge,genre_indie,genre_jazz,genre_metal,genre_pop,genre_punk,genre_rap,genre_rock
0,Gorillaz,On Melancholy Hill,0q6LuUqGLUiCPP1cbdwFs3,75,2010,alternative,0,0.000015,0.689,233867,0.739,0.509000,2,0.0640,-5.810,1,0.0260,120.423,4,0.578,1,0,0,0,0,0,0,0,0,0,0,0
1,Avenged Sevenfold,Nightmare,4UEo1b0wWrtHMC8bVqPiH8,70,2010,alternative,1,0.000318,0.554,374453,0.949,0.000100,2,0.2000,-4.928,1,0.0787,129.984,4,0.233,1,0,0,0,0,0,0,0,0,0,0,0
2,The Black Keys,Howlin' for You,0grFc6klR3hxoHLcgCYsF4,66,2010,alternative,2,0.028000,0.705,191800,0.735,0.078300,11,0.1120,-6.646,1,0.0931,132.627,4,0.448,1,0,0,0,0,0,0,0,0,0,0,0
3,My Darkest Days,Porn Star Dancing,3Q8zopc4ABXhysDb1sgLVW,65,2010,alternative,3,0.013900,0.477,199013,0.917,0.000000,2,0.0756,-3.399,1,0.0837,160.044,4,0.271,1,0,0,0,0,0,0,0,0,0,0,0
4,Volbeat,A Warrior's Call,0hTiTU0yqthnByyZDD3bcc,62,2010,alternative,4,0.000750,0.374,263080,0.903,0.000006,10,0.2440,-4.490,1,0.0825,109.118,3,0.429,1,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# show only the audio features.
genreframe.iloc[0,7:20]

acousticness        1.51e-05
danceability           0.689
duration_ms           233867
energy                 0.739
instrumentalness       0.509
key                        2
liveness               0.064
loudness               -5.81
mode                       1
speechiness            0.026
tempo                120.423
time_signature             4
valence                0.578
Name: 0, dtype: object

In [5]:
# set the X_train data on the audio features.
X_train = genreframe.iloc[:,7:20].to_numpy()
# set the standard scalar 
scaler = StandardScaler()
# fit the scalar on the X_train data.
X_scaled = scaler.fit_transform(X_train)
# show the X_scaled array.
X_scaled

array([[-0.81353437,  0.86273319,  0.01422737, ..., -0.07457545,
         0.21919423,  0.52020022],
       [-0.81253793,  0.02764277,  1.61395879, ...,  0.24680209,
         0.21919423, -0.95936143],
       [-0.72147348,  0.96170687, -0.46445403, ...,  0.33564226,
         0.21919423, -0.03731576],
       ...,
       [ 0.35424623,  1.02975127,  0.17080273, ..., -0.0196848 ,
         0.21919423,  1.42080297],
       [-0.80624809, -0.62187201, -1.57989421, ..., -1.07440547,
         0.21919423,  1.22781667],
       [-0.42211418,  1.23388449, -0.24267685, ..., -0.99696024,
         0.21919423, -0.50048289]])

In [6]:
# create a list of the genres.
genres = genreframe.genre.unique().tolist()
genres

['alternative',
 'country',
 'dance',
 'folk',
 'grunge',
 'indie',
 'jazz',
 'metal',
 'pop',
 'punk',
 'rap',
 'rock']

In [7]:
onehots = genreframe.iloc[:,20:]
y_train = onehots.to_numpy()
print(y_train.shape)
y_train

(164449, 12)


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [0]:
# function for NN model.
def create_model():
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    optimizer = Adam()

    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [10]:
# set the model.
model = create_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 26)                364       
_________________________________________________________________
dense_1 (Dense)              (None, 26)                702       
_________________________________________________________________
dense_2 (Dense)              (None, 26)                702       
_________________________________________________________________
dense_3 (Dense)              (None, 12)                324       
Total params: 2,092
Trainable params: 2,092
Non-trainable params: 0
_________________________________________________________________


In [12]:
# set the model.
model = create_model()
# fit the model with parameters.
model.fit(X_scaled, y_train, validation_split=0.2, batch_size=200, epochs=20)

Train on 131559 samples, validate on 32890 samples
Epoch 1/20
131559/131559 [==============================] - 2s 15us/sample - loss: 2.0056 - acc: 0.3072 - val_loss: 1.8977 - val_acc: 0.3657
Epoch 2/20
131559/131559 [==============================] - 2s 14us/sample - loss: 1.8784 - acc: 0.3505 - val_loss: 1.8780 - val_acc: 0.3760
Epoch 3/20
131559/131559 [==============================] - 2s 14us/sample - loss: 1.8594 - acc: 0.3572 - val_loss: 1.8702 - val_acc: 0.3749
Epoch 4/20
131559/131559 [==============================] - 2s 15us/sample - loss: 1.8479 - acc: 0.3613 - val_loss: 1.8597 - val_acc: 0.3798
Epoch 5/20
131559/131559 [==============================] - 2s 14us/sample - loss: 1.8394 - acc: 0.3640 - val_loss: 1.8574 - val_acc: 0.3814
Epoch 6/20
131559/131559 [==============================] - 2s 14us/sample - loss: 1.8326 - acc: 0.3653 - val_loss: 1.8383 - val_acc: 0.3892
Epoch 7/20
131559/131559 [==============================] - 2s 14us/sample - loss: 1.8279 - acc: 0.3672

In [0]:
# plot the accuracy vs epochs.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='center right')
# show the graph.
plt.show()

### Batch Size.

In [0]:
# function for NN model.
def create_model():
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
    return model

In [0]:
# set the model with kerasclassifier.
model = KerasClassifier(build_fn=create_model, verbose=0)

In [0]:
# set the parameters for batch size.
param_grid = {'batch_size': [10, 20, 30, 40, 50, 100]}

In [40]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.35273563861846924 using {'batch_size': 20}
Means: 0.3524437665939331, Stdev: 0.019404246883930283 with: {'batch_size': 10}
Means: 0.35273563861846924, Stdev: 0.019255442225379327 with: {'batch_size': 20}
Means: 0.3527234673500061, Stdev: 0.019018882278608488 with: {'batch_size': 30}
Means: 0.3503883719444275, Stdev: 0.018017765171301013 with: {'batch_size': 40}
Means: 0.3486066460609436, Stdev: 0.014345853511393053 with: {'batch_size': 50}
Means: 0.3463384985923767, Stdev: 0.015720864354716486 with: {'batch_size': 100}


### Epochs.

In [0]:
# function for NN model.
def create_model():
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
    return model

In [0]:
# set the model with kerasclassifier.
model = KerasClassifier(build_fn=create_model, batch_size=20, verbose=0)

In [0]:
# set the parameters for epochs.
param_grid = {'epochs': [10, 30, 50, 70, 100]}

In [0]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

### Optimizer.

In [0]:
# function for NN model.
def create_model(optimizer='adam'):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=1)

In [0]:
# set the parameters for optimizer.
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

In [0]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

### learning Rate.

In [0]:
# function for NN model.
def create_model(learn_rate=0.01):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(26, activation='relu'))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=learn_rate)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=0)

In [0]:
# set the parameters for learn rate.
learn_rate = [0, 0.001, 0.01, 0.05]
param_grid = dict(learn_rate=learn_rate)

In [20]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.360397607088089 using {'learn_rate': 0.001}
Means: 0.3572172403335571, Stdev: 0.017568420028037764 with: {'learn_rate': 0}
Means: 0.360397607088089, Stdev: 0.01969457336273759 with: {'learn_rate': 0.001}
Means: 0.3593577742576599, Stdev: 0.01843075745253979 with: {'learn_rate': 0.01}
Means: 0.3571929633617401, Stdev: 0.02148785370383387 with: {'learn_rate': 0.05}


### Activation Function.

In [0]:
# function for NN model.
def create_model(activation='relu'):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, activation=activation))
    model.add(Dense(26, activation=activation))
    model.add(Dense(26, activation=activation))
    model.add(Dense(12,activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=0)

In [0]:
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

In [92]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Best: 0.36251984238624574 using {'activation': 'relu'}
Means: 0.31171984076499937, Stdev: 0.019557800448349007 with: {'activation': 'softmax'}
Means: 0.35679763555526733, Stdev: 0.01782844723097705 with: {'activation': 'softplus'}
Means: 0.3545660197734833, Stdev: 0.0221664979022013 with: {'activation': 'softsign'}
Means: 0.36251984238624574, Stdev: 0.021896434306509477 with: {'activation': 'relu'}
Means: 0.35927869081497193, Stdev: 0.01796371880356253 with: {'activation': 'tanh'}
Means: 0.34020292162895205, Stdev: 0.020601240356198456 with: {'activation': 'sigmoid'}
Means: 0.34115758538246155, Stdev: 0.01922564049533639 with: {'activation': 'hard_sigmoid'}
Means: 0.33451111912727355, Stdev: 0.015171496764235657 with: {'activation': 'linear'}


### Weight Initialization.

In [0]:
# function for NN model.
def create_model(weight_init='uniform'):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, input_dim=13, kernel_initializer=weight_init, activation='relu'))
    model.add(Dense(26, kernel_initializer=weight_init, activation='relu'))
    model.add(Dense(26, kernel_initializer=weight_init, activation='relu'))
    model.add(Dense(12, kernel_initializer=weight_init, activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=0)

In [0]:
# define the grid search parameters
weight_init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(weight_init=weight_init)

In [28]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Best: 0.3589625060558319 using {'weight_init': 'he_uniform'}
Means: 0.3483147919178009, Stdev: 0.020422530207887157 with: {'weight_init': 'uniform'}
Means: 0.3555449903011322, Stdev: 0.01716633233750613 with: {'weight_init': 'lecun_uniform'}
Means: 0.3474938929080963, Stdev: 0.019531944385382388 with: {'weight_init': 'normal'}
Means: 0.1110739067196846, Stdev: 0.0037375406297293005 with: {'weight_init': 'zero'}
Means: 0.3584394991397858, Stdev: 0.018103023820388524 with: {'weight_init': 'glorot_normal'}
Means: 0.35727807879447937, Stdev: 0.01888037620712115 with: {'weight_init': 'glorot_uniform'}
Means: 0.35358694195747375, Stdev: 0.020085757183696422 with: {'weight_init': 'he_normal'}
Means: 0.3589625060558319, Stdev: 0.019940247827964698 with: {'weight_init': 'he_uniform'}


### Dropout Rate.

In [0]:
# function for NN model.
def create_model(weight_rate=0.0, weight_constraint=0):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(26, kernel_initializer='he_uniform', input_dim=13, activation='relu'))
    model.add(Dropout(weight_rate))
    model.add(Dense(26, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(weight_rate))
    model.add(Dense(26, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(weight_rate))
    model.add(Dense(12, kernel_initializer='he_uniform',activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=0)

In [0]:
# set the parameters for weights and dropout rate.
weight_constraint = [1, 2, 3, 4]
weight_rate = [0.0, 0.1, 0.2]
param_grid = dict(weight_rate=weight_rate, weight_constraint=weight_constraint)

In [33]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.3579287052154541 using {'weight_constraint': 4, 'weight_rate': 0.0}
Means: 0.3574301183223724, Stdev: 0.021968654631767656 with: {'weight_constraint': 1, 'weight_rate': 0.0}
Means: 0.35145255327224734, Stdev: 0.01747412935835445 with: {'weight_constraint': 1, 'weight_rate': 0.1}
Means: 0.3410906851291656, Stdev: 0.020216486745120475 with: {'weight_constraint': 1, 'weight_rate': 0.2}
Means: 0.35440181493759154, Stdev: 0.020200136275872138 with: {'weight_constraint': 2, 'weight_rate': 0.0}
Means: 0.3472020149230957, Stdev: 0.019681137337494343 with: {'weight_constraint': 2, 'weight_rate': 0.1}
Means: 0.3403123676776886, Stdev: 0.022672067006390642 with: {'weight_constraint': 2, 'weight_rate': 0.2}
Means: 0.35611668825149534, Stdev: 0.021626799691961622 with: {'weight_constraint': 3, 'weight_rate': 0.0}
Means: 0.3481810033321381, Stdev: 0.015999909472515082 with: {'weight_constraint': 3, 'weight_rate': 0.1}
Means: 0.34061639904975893, Stdev: 0.01979421526211651 with: {'weight_cons

### Hidden Neuron Count.

In [0]:
# function for NN model.
def create_model(neurons=1):
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(neurons, kernel_initializer='he_uniform', input_dim=13, activation='relu'))
    model.add(Dense(neurons, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(neurons, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(12, kernel_initializer='he_uniform', activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=20, verbose=0)

In [0]:
# set the parameters for neurons.
neurons = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
param_grid = dict(neurons=neurons)

In [45]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_scaled, y_train)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.36318874955177305 using {'neurons': 60}
Means: 0.11118336617946625, Stdev: 0.0035297070798743324 with: {'neurons': 1}
Means: 0.32387558221817014, Stdev: 0.021570116822359785 with: {'neurons': 5}
Means: 0.3442405939102173, Stdev: 0.016465594936106772 with: {'neurons': 10}
Means: 0.34894726276397703, Stdev: 0.019788778905189718 with: {'neurons': 15}
Means: 0.35209710001945493, Stdev: 0.017255614077919684 with: {'neurons': 20}
Means: 0.3558491051197052, Stdev: 0.02136150034797747 with: {'neurons': 25}
Means: 0.3582936108112335, Stdev: 0.020459841453516713 with: {'neurons': 30}
Means: 0.3617839813232422, Stdev: 0.01771471592677918 with: {'neurons': 35}
Means: 0.3595280110836029, Stdev: 0.022677723068950696 with: {'neurons': 40}
Means: 0.3562199532985687, Stdev: 0.01859260641970323 with: {'neurons': 45}
Means: 0.36123672127723694, Stdev: 0.016125944177393395 with: {'neurons': 50}
Means: 0.3609691560268402, Stdev: 0.014842394310193914 with: {'neurons': 55}
Means: 0.36318874955177305,

### Final Model.

In [0]:
# function for NN model.
def create_model():
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(768, kernel_initializer='he_uniform', input_dim=13, activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(384, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(192, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(96, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(48, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(24, kernel_initializer='he_uniform', activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(12, kernel_initializer='he_uniform', activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [0]:
# set the model.
model = create_model()
# fit the model with parameters.
model.fit(X_scaled, y_train, validation_split=0.2, epochs=200, batch_size=20)

Train on 131559 samples, validate on 32890 samples
Epoch 1/200
131559/131559 [==============================] - 24s 182us/sample - loss: 1.9288 - acc: 0.3382 - val_loss: 1.8567 - val_acc: 0.3860
Epoch 2/200
131559/131559 [==============================] - 24s 181us/sample - loss: 1.8558 - acc: 0.3621 - val_loss: 1.8425 - val_acc: 0.3882
Epoch 3/200
131559/131559 [==============================] - 23s 178us/sample - loss: 1.8328 - acc: 0.3687 - val_loss: 1.8477 - val_acc: 0.3884
Epoch 4/200
131559/131559 [==============================] - 23s 177us/sample - loss: 1.8198 - acc: 0.3726 - val_loss: 1.8867 - val_acc: 0.3685
Epoch 5/200
131559/131559 [==============================] - 23s 177us/sample - loss: 1.8096 - acc: 0.3766 - val_loss: 1.8676 - val_acc: 0.3763
Epoch 6/200
131559/131559 [==============================] - 23s 175us/sample - loss: 1.8009 - acc: 0.3780 - val_loss: 1.8539 - val_acc: 0.3721
Epoch 7/200
131559/131559 [==============================] - 23s 177us/sample - loss:

In [32]:
# final model accuracy score.
scores = model.evaluate(X_scaled, y_train)

164449/164449 [==============================] - 10s 58us/sample - loss: 1.6542 - acc: 0.4365


In [0]:
# plot the accuracy vs epochs.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='center right')
# show the graph.
plt.show()

In [0]:
# plot the loss vs epochs.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

### Results.

In [24]:
# Fit the model
history = model.fit(X_scaled, y_train, validation_split=0.2, epochs=10, batch_size=20)

Train on 131559 samples, validate on 32890 samples
Epoch 1/10
131559/131559 [==============================] - 23s 173us/sample - loss: 2.0464 - acc: 0.2907 - val_loss: 1.9089 - val_acc: 0.3711
Epoch 2/10
131559/131559 [==============================] - 21s 160us/sample - loss: 1.9631 - acc: 0.3235 - val_loss: 1.9196 - val_acc: 0.3627
Epoch 3/10
131559/131559 [==============================] - 21s 162us/sample - loss: 1.9490 - acc: 0.3292 - val_loss: 1.8866 - val_acc: 0.3784
Epoch 4/10
131559/131559 [==============================] - 21s 161us/sample - loss: 1.9410 - acc: 0.3310 - val_loss: 1.8837 - val_acc: 0.3804
Epoch 5/10
131559/131559 [==============================] - 21s 159us/sample - loss: 1.9366 - acc: 0.3336 - val_loss: 1.8660 - val_acc: 0.3800
Epoch 6/10
131559/131559 [==============================] - 21s 162us/sample - loss: 1.9327 - acc: 0.3355 - val_loss: 1.8635 - val_acc: 0.3849
Epoch 7/10
131559/131559 [==============================] - 21s 163us/sample - loss: 1.9298

In [0]:
obs = genreframe.iloc[10000:10001,7:20].to_numpy()
obs.shape

(1, 13)

In [0]:
model.predict(obs)

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [0]:
genreframe.iloc[10000:10001,5]

10000    jazz
Name: genre, dtype: object

In [0]:
model.save('genre_NN')

In [0]:
# function for NN model.
def create_model():
    # set the model.
    model = Sequential()
    # hidden layers.
    model.add(Dense(768, kernel_initializer='he_uniform', input_dim=13, activation='relu'))
    model.add(Dense(384, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(192, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(96, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(48, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(24, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(12, kernel_initializer='he_uniform', activation='softmax'))
    # function for top 3.
    def top_3_accuracy(y_true, y_pred):
        return top_k_categorical_accuracy(y_true, y_pred, k=3)

    # set the optimizer.
    optimizer = Adam(lr=0.001)
    # compile the model.
    model.compile(loss='categorical_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    return model

In [175]:
# set the model.
model = create_model()
# fit the model with parameters.
model.fit(X_scaled, y_train, validation_split=0.2, batch_size=200, epochs=200)

Train on 131559 samples, validate on 32890 samples
Epoch 1/1000
131559/131559 [==============================] - 14s 109us/sample - loss: 1.9262 - acc: 0.3401 - val_loss: 1.9097 - val_acc: 0.3618
Epoch 2/1000
131559/131559 [==============================] - 7s 53us/sample - loss: 1.8431 - acc: 0.3621 - val_loss: 1.8472 - val_acc: 0.3855
Epoch 3/1000
131559/131559 [==============================] - 7s 55us/sample - loss: 1.8197 - acc: 0.3712 - val_loss: 1.8299 - val_acc: 0.3896
Epoch 4/1000
131559/131559 [==============================] - 7s 53us/sample - loss: 1.8045 - acc: 0.3758 - val_loss: 1.8281 - val_acc: 0.3886
Epoch 5/1000
131559/131559 [==============================] - 7s 54us/sample - loss: 1.7937 - acc: 0.3785 - val_loss: 1.8217 - val_acc: 0.3891
Epoch 6/1000
131559/131559 [==============================] - 7s 54us/sample - loss: 1.7830 - acc: 0.3821 - val_loss: 1.8260 - val_acc: 0.3935
Epoch 7/1000
131559/131559 [==============================] - 7s 53us/sample - loss: 1.77

KeyboardInterrupt: ignored

In [13]:
# final model accuracy score.
scores = model.evaluate(X_scaled, y_train)

164449/164449 [==============================] - 8s 50us/sample - loss: 1.8027 - acc: 0.3798
